In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")

VERTEX_AI_KEY = os.getenv("GOOGLE_VERTEX_AI_KEY")
VERTEX_AI_PROJECT = os.getenv("GOOGLE_VERTEX_AI_PROJECT")

In [ ]:
VERTEX_AI_KEY = "ya29.a0AXooCgtJzUY8N1cF5O15Pjw48PfIYEq4l-4tNQGIJqLD9EtD-dVy9unVgoFaH4vWrmn4y6EenT2Nld4tPwk6Uw1iKZYyg21POksGB8jVJipKG8QcwLTHFgHJ6I06jLLn8xNUjk_-17o6ScoL0iDUZTGh2bnKfyyEPdGlKizTpjhTaCgYKAfgSARASFQHGX2Mi1c54QZegh2xs5YeAxsWp-Q0179"
# VERTEX_AI_KEY = "ya29.a0AXooCgskulIrPZyuV..."

## Connect with Vertex AI key

In [ ]:
import weaviate

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
        "X-Google-Api-Key": VERTEX_AI_KEY,
        "X-Google-Studio-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()

## Connect with CLIP

In [ ]:
import weaviate

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
        "X-Google-Studio-Api-Key": AI_STUDIO_KEY,
    }
)

## Search
### On `title` vector

In [ ]:
import weaviate.classes.query as wq

# Get the collection
movies = client.collections.get("MoviesMM")

# Perform query
response = movies.query.near_text(
    query="Movie about lovable cute pets",
    # query="Epic super hero",
    target_vector="content",  # The target vector to search against
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_url"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

### Get the BLOB object for the poster
By default, the Weaviate doesn't return blob objects, as they tend to be large, and it is a lot more efficient to skip them.

In order to get the `poster`, you need to use `return_properties` and request it along other required properties.

In [ ]:
import weaviate.classes.query as wq

# Get the collection
movies = client.collections.get("MoviesMM")

# Perform query
response = movies.query.near_text(
    query="Movie about lovable cute pets",
    # query="Epic super hero",
    target_vector="content",  # The content target vector
    limit=5,
    return_properties=["title", "release_date", "tmdb_id", "poster_path", "poster"], # return poster blob
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_path"],
    )

    # Print the poster blob
    print(o.properties["poster"])

    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

### Image search – On `poster` vector

In [ ]:
import weaviate.classes.query as wq

# Get the collection
movies = client.collections.get("MoviesMM")

# Perform query
response = movies.query.near_text(
    query="Epic super hero",
    target_vector="poster",  # The poster target vector
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_path"],
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

### `Image search` on `poster` vector

In [ ]:
from IPython.display import Image
display(Image("./test/mouse.jpg", width=150))
display(Image("./test/droid.jpeg", width=150))
display(Image("./test/spooky.jpg", width=250))

In [ ]:
import weaviate.classes.query as wq
from pathlib import Path

# Get the collection
movies = client.collections.get("MoviesMM")

# Perform query
response = movies.query.near_image(
    # near_image=Path("./test/mouse.jpg"),
    # near_image=Path("./test/droid.jpeg"),
    near_image=Path("./test/spooky.jpg"),
    target_vector="poster",  # The target vector to search against
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    display(Image(o.properties["poster_path"], width=200))

    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_path"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )